In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
import rioxarray as rxr
import glob
from datetime import datetime
import utm
from pyproj import CRS
import os

In [8]:
times = pd.read_csv('../../tifs/GOES_times.csv')

In [9]:
times.value = (times.value/1000).apply(datetime.fromtimestamp)

In [10]:
# Indexes with Landsat images:
Landsat_indexes = [0, 16, 32, 47, 62, 78, 94, 110]
times.value

0     2022-12-31 22:46:17.300
1     2023-01-01 10:46:17.400
2     2023-01-01 22:46:17.400
3     2023-01-02 10:46:17.400
4     2023-01-02 22:46:17.400
                ...          
720   2023-12-28 10:46:17.100
721   2023-12-28 22:46:17.100
722   2023-12-29 10:46:17.200
723   2023-12-29 22:46:17.200
724   2023-12-30 10:46:17.200
Name: value, Length: 725, dtype: datetime64[ns]

In [5]:
geotiff_list = glob.glob('../../tifs/*.tif')
geotiff_list

['../../tifs/GOES_Landsat_image_55.tif',
 '../../tifs/GOES_Landsat_image_41.tif',
 '../../tifs/GOES_Landsat_image_69.tif',
 '../../tifs/GOES_Landsat_image_103.tif',
 '../../tifs/GOES_Landsat_image_96.tif',
 '../../tifs/GOES_Landsat_image_82.tif',
 '../../tifs/GOES_Landsat_image_117.tif',
 '../../tifs/GOES_Landsat_image_7.tif',
 '../../tifs/GOES_Landsat_image_6.tif',
 '../../tifs/GOES_Landsat_image_83.tif',
 '../../tifs/GOES_Landsat_image_116.tif',
 '../../tifs/GOES_Landsat_image_102.tif',
 '../../tifs/GOES_Landsat_image_97.tif',
 '../../tifs/GOES_Landsat_image_68.tif',
 '../../tifs/GOES_Landsat_image_40.tif',
 '../../tifs/GOES_Landsat_image_54.tif',
 '../../tifs/GOES_Landsat_image_42.tif',
 '../../tifs/GOES_Landsat_image_56.tif',
 '../../tifs/GOES_Landsat_image_114.tif',
 '../../tifs/GOES_Landsat_image_81.tif',
 '../../tifs/GOES_Landsat_image_95.tif',
 '../../tifs/GOES_Landsat_image_100.tif',
 '../../tifs/GOES_Landsat_image_4.tif',
 '../../tifs/GOES_Landsat_image_5.tif',
 '../../tifs/G

In [6]:
def sort_func(s):
    return int(s.split('image_')[1].split('.tif')[0])

In [7]:
geotiff_list = sorted(geotiff_list, key=sort_func)
geotiff_list

['../../tifs/GOES_Landsat_image_0.tif',
 '../../tifs/GOES_Landsat_image_1.tif',
 '../../tifs/GOES_Landsat_image_2.tif',
 '../../tifs/GOES_Landsat_image_3.tif',
 '../../tifs/GOES_Landsat_image_4.tif',
 '../../tifs/GOES_Landsat_image_5.tif',
 '../../tifs/GOES_Landsat_image_6.tif',
 '../../tifs/GOES_Landsat_image_7.tif',
 '../../tifs/GOES_Landsat_image_8.tif',
 '../../tifs/GOES_Landsat_image_9.tif',
 '../../tifs/GOES_Landsat_image_10.tif',
 '../../tifs/GOES_Landsat_image_11.tif',
 '../../tifs/GOES_Landsat_image_12.tif',
 '../../tifs/GOES_Landsat_image_13.tif',
 '../../tifs/GOES_Landsat_image_14.tif',
 '../../tifs/GOES_Landsat_image_15.tif',
 '../../tifs/GOES_Landsat_image_16.tif',
 '../../tifs/GOES_Landsat_image_17.tif',
 '../../tifs/GOES_Landsat_image_18.tif',
 '../../tifs/GOES_Landsat_image_19.tif',
 '../../tifs/GOES_Landsat_image_20.tif',
 '../../tifs/GOES_Landsat_image_21.tif',
 '../../tifs/GOES_Landsat_image_22.tif',
 '../../tifs/GOES_Landsat_image_23.tif',
 '../../tifs/GOES_Landsat_

In [8]:
def UTM_to_lat_lon(easting_array, northing_array, EPSG):
    crs = CRS.from_user_input(EPSG)
    zone = int(crs.coordinate_operation.name[-3:-1])
    north = crs.coordinate_operation.name[-1:] == 'N'

    latitude = []
    longitude = []

    for val in easting_array:
        lon = utm.to_latlon(easting=val, northing=0, zone_number=zone, northern=north)[1].values
        longitude.append(lon)
    for val in northing_array:
        lat = utm.to_latlon(easting=100000, northing=val, zone_number=zone, northern=north)[0].values
        latitude.append(lat)

    return latitude, longitude

In [9]:
def to_binary_string(num):
    if np.isnan(num):
        return np.NaN
    else:
        return f'{int(num):b}'

In [10]:
to_binary_string(21952)

'101010111000000'

In [59]:
"""
Processing of individual .tif files.

Performs a variety of tasks on the data to make it more easy to read and understand.

Attributes:
    tif (str): Path where tif file is located.
    time (datetime or str): Date and time of when the data was collected.
    name (str): Desired name of output file. Just the name, leave out the ".format" part at the end.
    coord_bounds (tuple or list, optional): Coordinate bounds if you wish to filter the data by location. Order should be
                                    (longitude minimum, longitude maximum, latitude minimum, latitude maximum).
    save_as (str, optional): Format to save the processed data in. For now, only 'nc' is built-in. Anything else
                                    simply returns the data.
"""
def process_tif(tif, time, name, coord_bounds=None, save_as='nc'):
    #########################################################################################################
    # Open file and rename variables
    ds = rxr.open_rasterio(tif)
    geotiff_ds = ds.to_dataset('band')
    geotiff_ds = geotiff_ds.rename({1:'Landsat_Blue_Vis_Sfc_Reflectance', 2:'Landsat_Red_Vis_Sfc_Reflectance',
                                3:'Landsat_NIR_Sfc_Reflectance', 4:'Landsat_SWIR1_Sfc_Reflectance',
                               5:'Landsat_SWIR2_Sfc_Reflectance', 6:'Landsat_LST',
                               7:'Landsat_Cloud_Mask', 8:'GOES_Blue_Vis_Reflectance',
                               9:'GOES_Red_Vis_Reflectance', 10:'GOES_NIR_Veggie_Reflectance',
                               11:'GOES_NIR_SnowIce_Reflectance', 12:'GOES_NIR_CloudParticle_Reflectance',
                               13:'GOES_LWIR_Brightness_Temp', 14:'GOES_Dirty_LWIR_Brightness_temp'})

    #########################################################################################################
    # Convert UTM coordinates to latitude and longitude
    #latitude, longitude = UTM_to_lat_lon(geotiff_ds.x, geotiff_ds.y, 32618)
    #geotiff_ds['x'] = longitude
    #geotiff_ds['y'] = latitude
    #geotiff_ds = geotiff_ds.rename({'x':'longitude', 'y':'latitude'})

    #########################################################################################################
    # Convert cloud mask integers to binary strings
    mask = xr.DataArray([[to_binary_string(x) for x in line] for line in geotiff_ds.Landsat_Cloud_Mask.values])
    mask = mask.rename({'dim_0':'y', 'dim_1':'x'})
    geotiff_ds['Landsat_Cloud_Mask'] = mask

    #########################################################################################################
    # Assign descriptive attributes for the cloud mask and the file datetime
    geotiff_ds = geotiff_ds.assign_attrs(
        Cloud_Mask_Bits="Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high confidence)\nBit 3: Cloud\nBit 4: Cloud Shadow\n\
    Bit 5: Snow\nBit 6: Clear\n    0: Cloud or Dilated Cloud bits are set\n    1: Cloud and Dilated Cloud bits are not set\nBit 7: Water\n\
    Bits 8-9: Cloud Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 10-11: Cloud Shadow Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 12-13: Snow/Ice Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 14-15: Cirrus Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High")

    geotiff_ds = geotiff_ds.assign_attrs(Datetime=str(time))

    #########################################################################################################
    # Optional filtering by lat/lon
    if coord_bounds:
        geotiff_ds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

    #########################################################################################################
    if save_as=='nc':
        # Convert file into netCDF
        geotiff_ds.to_netcdf(f'{name}.nc')
    else:
        return geotiff_ds

In [61]:
# Runs processing function on all the geotif files
coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

for i in Landsat_indexes:
    process_tif(geotiff_list[i], times.value[i], f'../../processed2/tif_{i}')

## Sample Code

In [11]:
ds = rxr.open_rasterio('../../tifs/Landsat_Sentinel_image_0.tif')

In [12]:
geotiff_ds = ds.to_dataset('band')

In [13]:
geotiff_ds

<xarray.Dataset>
Dimensions:      (y: 3000, x: 3000)
Coordinates:
  * x            (x) float64 2.92e+05 2.92e+05 2.921e+05 ... 3.82e+05 3.82e+05
  * y            (y) float64 4.372e+06 4.372e+06 ... 4.282e+06 4.282e+06
    spatial_ref  int64 0
Data variables:
    1            (y, x) float64 ...
    2            (y, x) float64 ...
    3            (y, x) float64 ...
    4            (y, x) float64 ...
    5            (y, x) float64 ...
    6            (y, x) float64 ...
    7            (y, x) float64 ...
    8            (y, x) float64 ...
    9            (y, x) float64 ...
    10           (y, x) float64 ...
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [6]:
# For GOES tifs
geotiff_ds = geotiff_ds.rename({1:'GOES_Blue_Vis_Reflectance',
                               2:'GOES_Red_Vis_Reflectance', 3:'GOES_NIR_Veggie_Reflectance',
                               4:'GOES_NIR_SnowIce_Reflectance', 5:'GOES_NIR_CloudParticle_Reflectance',
                               6:'GOES_LWIR_Brightness_Temp', 7:'GOES_Dirty_LWIR_Brightness_temp'})

In [14]:
# For Landsat/Sentinel tifs
geotiff_ds = geotiff_ds.rename({1:'Landsat_Blue_Vis_Sfc_Reflectance', 2:'Landsat_Red_Vis_Sfc_Reflectance',
                                3:'Landsat_NIR_Sfc_Reflectance', 4:'Landsat_SWIR1_Sfc_Reflectance',
                               5:'Landsat_SWIR2_Sfc_Reflectance', 6:'Landsat_LST',
                               7:'Landsat_Cloud_Mask', 8:'VV_SAR', 9:'VH_SAR', 10:'SAR_angle'})

In [15]:
geotiff_ds

<xarray.Dataset>
Dimensions:                           (y: 3000, x: 3000)
Coordinates:
  * x                                 (x) float64 2.92e+05 2.92e+05 ... 3.82e+05
  * y                                 (y) float64 4.372e+06 ... 4.282e+06
    spatial_ref                       int64 0
Data variables:
    Landsat_Blue_Vis_Sfc_Reflectance  (y, x) float64 ...
    Landsat_Red_Vis_Sfc_Reflectance   (y, x) float64 ...
    Landsat_NIR_Sfc_Reflectance       (y, x) float64 ...
    Landsat_SWIR1_Sfc_Reflectance     (y, x) float64 ...
    Landsat_SWIR2_Sfc_Reflectance     (y, x) float64 ...
    Landsat_LST                       (y, x) float64 ...
    Landsat_Cloud_Mask                (y, x) float64 ...
    VV_SAR                            (y, x) float64 ...
    VH_SAR                            (y, x) float64 ...
    SAR_angle                         (y, x) float64 ...
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [16]:
geotiff_ds.y[0]

<xarray.DataArray 'y' ()>
array(4372185.)
Coordinates:
    y            float64 4.372e+06
    spatial_ref  int64 0

In [17]:
geotiff_ds.x[0]

<xarray.DataArray 'x' ()>
array(292015.)
Coordinates:
    x            float64 2.92e+05
    spatial_ref  int64 0

In [27]:
geotiff_ds.VH_SAR.values

array([[         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       ...,
       [-14.35910724, -12.73652802, -16.44185505, ..., -32.73728687,
        -32.06150714, -31.05807109],
       [-13.27172868, -14.002548  , -10.06917126, ..., -31.87240523,
        -31.12023307, -33.45023946],
       [-11.79934787, -16.03655774, -11.13412762, ..., -30.71765902,
        -31.4397387 , -32.93874892]])

In [32]:
#latitude, longitude = UTM_to_lat_lon(geotiff_ds.x, geotiff_ds.y, 32618)

In [33]:
#np.min(latitude), np.max(latitude), np.min(longitude), np.max(longitude)

In [34]:
#geotiff_ds['x'] = longitude
#geotiff_ds['y'] = latitude

In [35]:
#geotiff_ds = geotiff_ds.rename({'x':'longitude', 'y':'latitude'})

In [119]:
coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

In [36]:
#gds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

In [45]:
mask = xr.DataArray([[to_binary_string(x) for x in line] for line in geotiff_ds.Landsat_Cloud_Mask.values])

In [46]:
mask

<xarray.DataArray (dim_0: 3000, dim_1: 3000)>
array([['1101011100001100', '1101011100001100', '1101011100001100', ...,
        '101010101000000', '101010101000000', '101011001000000'],
       ['1101011100001100', '1101011100001100', '1101011100001100', ...,
        '101010101000000', '101011001000000', '101010101000000'],
       ['1101011100001100', '1101011100001100', '1101011100001100', ...,
        '101010101000000', '101010101000000', '101010101000000'],
       ...,
       ['101010101000000', '101010101000000', '101010101000000', ...,
        '101010111000000', '101010111000000', '101010111000000'],
       ['101010101000000', '101010101000000', '101010101000000', ...,
        '101010111000000', '101010111000000', '101010111000000'],
       ['101010101000000', '101010101000000', '101010101000000', ...,
        '101010111000000', '101010111000000', '101010111000000']],
      dtype='<U16')
Dimensions without coordinates: dim_0, dim_1

In [48]:
water = np.array([[s[-8] for s in line] for line in mask.values])

In [49]:
wmask = (water == '0')*1

In [50]:
np.sum((geotiff_ds.Landsat_LST)*wmask == 0)

<xarray.DataArray 'Landsat_LST' ()>
array(993622)
Coordinates:
    spatial_ref  int64 0

In [47]:
mask = mask.rename({'dim_0':'y', 'dim_1':'x'})

In [52]:
geotiff_ds['Landsat_Cloud_Mask'] = mask

In [54]:
geotiff_ds

<xarray.Dataset>
Dimensions:                             (y: 3000, x: 3000)
Coordinates:
  * x                                   (x) float64 2.92e+05 ... 3.82e+05
  * y                                   (y) float64 4.372e+06 ... 4.282e+06
    spatial_ref                         int64 0
Data variables: (12/14)
    Landsat_Blue_Vis_Sfc_Reflectance    (y, x) float64 ...
    Landsat_Red_Vis_Sfc_Reflectance     (y, x) float64 ...
    Landsat_NIR_Sfc_Reflectance         (y, x) float64 ...
    Landsat_SWIR1_Sfc_Reflectance       (y, x) float64 ...
    Landsat_SWIR2_Sfc_Reflectance       (y, x) float64 ...
    Landsat_LST                         (y, x) float64 255.9 255.9 ... 279.1
    ...                                  ...
    GOES_Red_Vis_Reflectance            (y, x) float64 ...
    GOES_NIR_Veggie_Reflectance         (y, x) float64 ...
    GOES_NIR_SnowIce_Reflectance        (y, x) float64 ...
    GOES_NIR_CloudParticle_Reflectance  (y, x) float64 ...
    GOES_LWIR_Brightness_Temp           (y, x) float64 ...
    GOES_Dirty_LWIR_Brightness_temp     (y, x) float64 ...
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [55]:
geotiff_ds = geotiff_ds.assign_attrs(
    Cloud_Mask_Bits="Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high confidence)\nBit 3: Cloud\nBit 4: Cloud Shadow\n\
Bit 5: Snow\nBit 6: Clear\n    0: Cloud or Dilated Cloud bits are set\n    1: Cloud and Dilated Cloud bits are not set\nBit 7: Water\n\
Bits 8-9: Cloud Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 10-11: Cloud Shadow Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 12-13: Snow/Ice Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 14-15: Cirrus Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High")

In [56]:
geotiff_ds = geotiff_ds.assign_attrs(Datetime=str(times.value[0]))

In [57]:
geotiff_ds

<xarray.Dataset>
Dimensions:                             (y: 3000, x: 3000)
Coordinates:
  * x                                   (x) float64 2.92e+05 ... 3.82e+05
  * y                                   (y) float64 4.372e+06 ... 4.282e+06
    spatial_ref                         int64 0
Data variables: (12/14)
    Landsat_Blue_Vis_Sfc_Reflectance    (y, x) float64 ...
    Landsat_Red_Vis_Sfc_Reflectance     (y, x) float64 ...
    Landsat_NIR_Sfc_Reflectance         (y, x) float64 ...
    Landsat_SWIR1_Sfc_Reflectance       (y, x) float64 ...
    Landsat_SWIR2_Sfc_Reflectance       (y, x) float64 ...
    Landsat_LST                         (y, x) float64 255.9 255.9 ... 279.1
    ...                                  ...
    GOES_Red_Vis_Reflectance            (y, x) float64 ...
    GOES_NIR_Veggie_Reflectance         (y, x) float64 ...
    GOES_NIR_SnowIce_Reflectance        (y, x) float64 ...
    GOES_NIR_CloudParticle_Reflectance  (y, x) float64 ...
    GOES_LWIR_Brightness_Temp           (y, x) float64 ...
    GOES_Dirty_LWIR_Brightness_temp     (y, x) float64 ...
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2024-01-14 10:46:17.300000

In [ ]:
# Export the data
#geotiff_ds.rio.to_raster("first.tif")
geotiff_ds.to_netcdf('first.nc')

In [27]:
xr.load_dataset('first.nc')

<xarray.Dataset>
Dimensions:                             (latitude: 6400, longitude: 6400)
Coordinates:
  * longitude                           (longitude) float64 -77.51 ... -75.79
  * latitude                            (latitude) float64 39.9 39.9 ... 38.17
Data variables: (12/15)
    Landsat_Blue_Vis_Sfc_Reflectance    (latitude, longitude) float64 nan ......
    Landsat_Red_Vis_Sfc_Reflectance     (latitude, longitude) float64 nan ......
    Landsat_NIR_Sfc_Reflectance         (latitude, longitude) float64 nan ......
    Landsat_SWIR1_Sfc_Reflectance       (latitude, longitude) float64 nan ......
    Landsat_SWIR2_Sfc_Reflectance       (latitude, longitude) float64 nan ......
    Landsat_LST                         (latitude, longitude) float64 nan ......
    ...                                  ...
    GOES_NIR_Veggie_Reflectance         (latitude, longitude) float64 0.1299 ...
    GOES_NIR_SnowIce_Reflectance        (latitude, longitude) float64 0.06935...
    GOES_NIR_CloudParticle_Reflectance  (latitude, longitude) float64 0.06691...
    GOES_LWIR_Brightness_Temp           (latitude, longitude) float64 274.2 ....
    GOES_Dirty_LWIR_Brightness_temp     (latitude, longitude) float64 273.1 ....
    spatial_ref                         int64 0
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:33.089500

In [ ]:
######################################################
# End of tif processing part
# Below is gridding of Baltimore

In [31]:
grid_lats = [39.2576, 39.3147, 39.3718]
grid_lons = [-76.5299, -76.6204, -76.7108]

In [37]:
grid1 = geotiff_ds.sel(longitude=slice(grid_lons[2], grid_lons[1])).sel(latitude=slice(grid_lats[2], grid_lats[1]))
grid2 = geotiff_ds.sel(longitude=slice(grid_lons[1], grid_lons[0])).sel(latitude=slice(grid_lats[2], grid_lats[1]))
grid3 = geotiff_ds.sel(longitude=slice(grid_lons[2], grid_lons[1])).sel(latitude=slice(grid_lats[1], grid_lats[0]))
grid4 = geotiff_ds.sel(longitude=slice(grid_lons[1], grid_lons[0])).sel(latitude=slice(grid_lats[1], grid_lats[0]))

In [38]:
grid4

<xarray.Dataset>
Dimensions:                             (latitude: 212, longitude: 336)
Coordinates:
  * longitude                           (longitude) float64 -76.62 ... -76.53
  * latitude                            (latitude) float64 39.31 39.31 ... 39.26
    spatial_ref                         int64 0
Data variables: (12/14)
    Landsat_Blue_Vis_Sfc_Reflectance    (latitude, longitude) float64 ...
    Landsat_Red_Vis_Sfc_Reflectance     (latitude, longitude) float64 ...
    Landsat_NIR_Sfc_Reflectance         (latitude, longitude) float64 ...
    Landsat_SWIR1_Sfc_Reflectance       (latitude, longitude) float64 ...
    Landsat_SWIR2_Sfc_Reflectance       (latitude, longitude) float64 ...
    Landsat_LST                         (latitude, longitude) float64 ...
    ...                                  ...
    GOES_Red_Vis_Reflectance            (latitude, longitude) float64 ...
    GOES_NIR_Veggie_Reflectance         (latitude, longitude) float64 ...
    GOES_NIR_SnowIce_Reflectance        (latitude, longitude) float64 ...
    GOES_NIR_CloudParticle_Reflectance  (latitude, longitude) float64 ...
    GOES_LWIR_Brightness_Temp           (latitude, longitude) float64 ...
    GOES_Dirty_LWIR_Brightness_temp     (latitude, longitude) float64 ...
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:33.089500